## 爬虫

### 响应response

一般我们得到的响应数据有两种：

- `html`

- `json`，书写格式类似于python的字典 

之后我们需要解析网页数据才能将干净的数据存储到数据库或者相应的文件中

**比较有代表性的是**

[**豆瓣读书 https://book.douban.com/tag/小说**](https://book.douban.com/tag/小说)

有的网站，我们在页面上看到的内容存在于json文件，而不是html文件。

**json文件非常类似于python的字典数据类型**

```json
{
"sites": [
          { "name":"百度" , "url":"www.baidu.com" },
          { "name":"google" , "url":"www.google.com" },
          { "name":"微博" , "url":"www.weibo.com" }
         ]
}
```


### 开发者工具

- ``Element面板``：查看网页元素（html网页数据）
- ``Network面板``：查看浏览器与网站之间的数据交互信息，多用于**抓包**


#### 批量生成网址

In [ ]:
template = 'https://book.douban.com/tag/小说?start={param}&type=T'
import time

for page in range(1, 11):
    url = template.format(param=(page-1)*20)
    print('第{}页url:'.format(page), url)
    time.sleep(1)

 
## 分析网址规律

1. 网址包含信息

2. 时候需要load `javascript`

3. response是否位json数据

4. 是否需要用模拟浏览器

## requests库

In [ ]:
# !pip3 install requests

找到网页url规律后，我们需要将这些url对应的网页数据下载下来，这里就用到[**requests库文档链接**](http://docs.python-requests.org/zh_CN/latest/user/quickstart.html)。

**requests两种访问方法** ,两者都返回Response对象：

|requests常用函数|参数解读|
|:---|:---|
|**requests.get(url, params, verify)** |发起get访问，返回**Response对象**；除非需要传参，否则不需要用**params和verify参数**| 
|**requests.post(url, data)** |发起post访问，返回**Response对象**；除非需要传参，否则不需要用**data参数**| 

In [ ]:
import requests

url = 'https://book.douban.com/tag/小说?start=0&type=T'
resp = requests.get(url)  #极少数情况会用到verify=False这参数
resp

**注意Response后面带有的状态码：**
- 2开头表示访问正常
- 4开头，比如403表示爬虫被网站封锁
- 5开头表示服务器出问题

### Response响应对象

|**Response对象的方法**|作用|
|:---|:---|
|**Response.json()**|获得json格式网页数据|
|**Response.text**|获得html网页数据|
|**Response.content**|获得网页**二进制文件数据**，常常用该方法爬取**图片、视频**等数据|
|**Response.encoding**|更改网页数据的编码方式。除非使用Response.text得到的html中文本是乱码的，否则一般不用Response.encoding|
|**Response.status_code**|查看当前访问的状态码， ``200`` 表示访问正常， ``4开头`` 的状态码表示访问出问题， ``5开头`` 的状态码表示服务器出问题|

以上三种方法，最常用的是 ``text和json方法``， 分别对应于 ``html网页数据和json网页数据``

In [ ]:
resp.text[:200]

In [ ]:
resp.encoding 

In [ ]:
# resp.json()

这里经过抓包分析（**爬虫最重要的就是抓包分析**）找到页面对应的url为

```
https://movie.douban.com/j/new_search_subjects?sort=U&range=0,10&tags=&start=40
```

In [ ]:
import requests

url = 'https://movie.douban.com/j/new_search_subjects?sort=U&range=0,10&tags=&start=40'
resp = requests.get(url)

resp.json()

In [ ]:
import requests

url_template = 'https://movie.douban.com/j/new_search_subjects?sort=U&range=0,10&tags=&start={num}'

for p in range(1, 10):
    url = url_template.format(num=(p-1)*20)
    print(url)

## requests.get()参数

### headers - 将爬虫伪装成浏览器
为了防止被网站封锁，我们需要将请求头headers加进访问信息中，从而将爬虫伪装成浏览器访问。

最常见的headers只有一个**User-Agent参数**即可,**让网站误以为是Chrome浏览器在访问网页数据**。

```python
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

url = '要访问网页的url'

r  = requests.get(url, headers = headers)
```

**[大众点评杭州酒店页面](http://www.dianping.com/hangzhou/hotel)** 、 http://www.dianping.com/hangzhou/hotel

In [ ]:
import requests

url = 'http://www.dianping.com/hangzhou/hotel/p4'
req = requests.get(url)

#403，禁止我们爬虫访问
req.text

In [ ]:
import requests

url = 'http://www.dianping.com/hangzhou/hotel'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'}
req = requests.get(url, headers=headers)

#200，经过简单伪装后的爬虫能正常访问
req.text

## 模拟浏览器

一般的写爬虫的方法是用python脚本直接对目标网站进行访问，而且只对目标数据进行采集，访问速度很快，这样目标网站很容易就识别出你是机器人，然后把你封锁了~~

**而**

使用selenium写爬虫，python脚本操控浏览器进行访问，也就是说python脚本和目标网站之间多了个浏览器的操作，这样的行为更像是人类行为。这样很多难爬的网站也可以轻而易举的抓数据了。

-----

**selenium支持**

- chromedriver，可操控chrome浏览器 [下载地址](https://chromedriver.chromium.org/downloads)

- Firefoxdriver ，可操控Firefox浏览器 [下载地址](https://chromedriver.chromium.org/downloads)

In [ ]:
# !pip3 install selenium

In [ ]:
from selenium import webdriver
import time

#初始化浏览器
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.baidu.com/')

driver.find_element_by_xpath('//*[@id="kw"]').clear()
driver.find_element_by_xpath('//*[@id="kw"]').send_keys('暨南大学\n')

In [ ]:
driver.find_element_by_xpath('//*[@id="1"]/h3/a[1]').click()

In [ ]:
p = driver.page_source

In [ ]:
type(p)

## 解析HTML 

In [ ]:
html_string = open("jnu-homepage/暨南大学.html").read()

In [ ]:
import requests

url = 'https://book.douban.com/tag/小说?start=0&type=T'
resp = requests.get(url)  #极少数情况会用到verify=False这参数
resp

In [ ]:
html_string = resp.text # local html

In [ ]:
type(html_string)

In [ ]:
from pyquery import PyQuery
doc = PyQuery(html_string)
print(type(doc))

为了定位html中对应的节点及其属性和含有的信息，我们需要使用选择器。



|css选择器|例子|解释|
|---|---|---|
|``.class``|``.intro``|选出``class="intro"``的节点|
|``#id``|``#firstname``|选出``id="firstname"``的节点|
|element|p|选出所有p标签的节点|
|element element|``div p``|选出div节点后辈p的所有节点|
|``*``|``#firstname``|选出``id="firstname"``的节点|
|``[attribute]``|``[target]``|选出带有 target 属性所有节点|
|``[attribute=value]``|``[target=_blank]``|选出 ``target="_blank"`` 的所有节点|


常用的css选择器主要有 .class  #id  element 这三种，本文只讲class、id 和lement最常见的css选择器。

In [ ]:
div = doc("#subject_list > ul > li:nth-child(1)")
print(div)

In [ ]:
div = doc(".subject-item")
len(div)

In [ ]:
for item in div.items():
    print(type(item), item.text() , sep = "\t")

In [ ]:
div = next(div.items())

In [ ]:
print(div)

In [ ]:
print(div("a[title]").text())

In [ ]:
div(".pub").text()

In [ ]:
print(div(".rating_nums").text())

In [ ]:
print(div(".pl").text())

In [ ]:
url = div("img").attr('src')
url

In [ ]:
from urllib.request import urlretrieve

In [ ]:
local = url.split("/")[-1]
local

In [ ]:
urlretrieve(url, local)

## 爬取企业信用数据

In [ ]:
from selenium import webdriver
from pyquery import PyQuery
import time
import re
import pandas as pd


#存储数据
csvf = open('companys.csv', 'w', # encoding='gbk', 
            newline='')
writer = csv.writer(csvf)
writer.writerow(('name', 'law_person', 'capital', 'register_date', 'address'))

#初始化浏览器
driver = webdriver.Chrome(
    #executable_path='driver/chromedriver'
)

url = 'https://xin.baidu.com/s?q=家化&t=0'
#打开某个网址
driver.get(url)
time.sleep(2)

for page in range(2, 9):
    pagexpath = '/html/body/div[2]/div/div[2]/div[5]/div[2]/div/a[{p}]'.format(p=page)
        #定位输入密码框，清除框内信息，再输入你的密码
    driver.find_element_by_xpath(pagexpath).click()
    time.sleep(1)

        #获取该网页的源码
    html = driver.page_source
    doc = PyQuery(html)
    companys = doc.items('.zx-ent-info')
    for company in companys:
        name = company.find('.zx-list-item-url').attr('title')
        law_person = company.find('.legal-txt').attr('title')
        capital = str(company.find(".zx-ent-pre-title").eq(1)).split('>')[-1]
        register_date = str(company.find(".zx-ent-pre-title").eq(2)).split('>')[-1]
        address = str(company.find(".zx-ent-pre-title").eq(3)).split('>')[-1]
        writer.writerow((name, law_person, capital, register_date, address))
        print(name, law_person, capital, register_date, address)

csvf.close()